In [666]:
#EDA for deriving Expected EAR and Actual EAR

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.feature_selection import VarianceThreshold
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from tqdm.auto import tqdm
tqdm.pandas()
warnings.filterwarnings('ignore')

data columns

In [35]:
column_info = pd.read_csv('HomeCredit_columns_description.csv', encoding = 'unicode_escape')

In [37]:
column_info[column_info['Table'] == 'previous_application.csv'].head(5)

,Unnamed: 0,Table,Row,Description,Special
173,176,previous_application.csv,SK_ID_PREV,ID of previous credit in Home credit related t...,hashed
174,177,previous_application.csv,SK_ID_CURR,ID of loan in our sample,hashed
175,178,previous_application.csv,NAME_CONTRACT_TYPE,"Contract product type (Cash loan, consumer loa...",NaN
176,179,previous_application.csv,AMT_ANNUITY,Annuity of previous application,NaN
177,180,previous_application.csv,AMT_APPLICATION,For how much credit did client ask on the prev...,NaN


In [600]:
import pandas as pd
import numpy as np

In [601]:
payments_file = 'installments_payments.csv'
previous_file = 'previous_application.csv'
df_payments = pd.read_csv(payments_file)
df_previous = pd.read_csv(previous_file)

In [602]:
(df_previous.shape, df_payments.shape)

((1670214, 37), (13605401, 8))

In [603]:
df_previous_filtered = df_previous[['SK_ID_CURR','SK_ID_PREV','AMT_ANNUITY','AMT_CREDIT','CNT_PAYMENT']]
#[df_previous['SK_ID_CURR'] == 100042]

In [604]:
df_previous_filtered.shape

(1670214, 5)

In [605]:
#df_previous_filtered[['AMT_ANNUITY','AMT_CREDIT','CNT_PAYMENT']]

In [606]:
del df_previous
gc.collect()

128

In [607]:
df_previous_filtered['AMT_ANNUITY'] = df_previous_filtered['AMT_ANNUITY'].fillna(0)
df_previous_filtered['AMT_CREDIT'] = df_previous_filtered['AMT_CREDIT'].fillna(0)
df_previous_filtered['CNT_PAYMENT'] = df_previous_filtered['CNT_PAYMENT'].fillna(0)

In [608]:
def get_ear(annuity, credit, no_of_payments):
    """ derives ear and returns
    """
    PMT = [-credit]
    PMT = PMT + ([annuity] * int(no_of_payments))
    irr = np.irr(PMT)
    
    return (1+irr)**12 - 1

In [609]:
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered\
.progress_apply(lambda x: get_ear(x.AMT_ANNUITY, x.AMT_CREDIT, x.CNT_PAYMENT), axis=1)

In [610]:
df_previous_filtered['EXPECTED_EAR'] = df_previous_filtered['EXPECTED_EAR'].fillna(0)

In [611]:
#df_previous_filtered[df_previous_filtered['EXPECTED_EAR'].isnull()]

In [612]:
df_expected_ear = df_previous_filtered.groupby('SK_ID_CURR')['EXPECTED_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_EXPECTED_EAR','median':'MEDIAN_EXPECTED_EAR', 'std':'STD_EXPECTED_EAR'})

In [613]:
#del df_previous_filtered
#gc.collect()

In [614]:
df_expected_ear.head(6)

,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,
100001,1.199400,1.199400,NaN
100002,0.238453,0.238453,NaN
100003,0.368710,0.385640,0.083249
100004,0.360515,0.360515,NaN
100005,0.516463,0.516463,0.730389
100006,0.282858,0.342949,0.294920


In [620]:
df_expected_ear['MEAN_EXPECTED_EAR'] = df_expected_ear['MEAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['MEDIAN_EXPECTED_EAR'] = df_expected_ear['MEDIAN_EXPECTED_EAR'].fillna(0)
df_expected_ear['STD_EXPECTED_EAR'] = df_expected_ear['STD_EXPECTED_EAR'].fillna(0)

In [621]:
#df_previous_payments = df_previous.merge(df_payments, on='SK_ID_CURR', how='inner')

In [622]:
#df_previous_payments.shape

In [623]:
#df_previous_10 = df_previous[df_previous['SK_ID_PREV'] == 2604725]

In [624]:
df_payments_filtered = df_payments[['SK_ID_PREV','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT']][df_payments['SK_ID_PREV'].isin(df_previous_filtered['SK_ID_PREV'].tolist())]
#df_payments_filtered = df_payments[['SK_ID_PREV','DAYS_INSTALMENT','DAYS_ENTRY_PAYMENT','AMT_INSTALMENT']][df_payments['SK_ID_PREV'].isin(['1125423'])]

In [625]:
df_payments_filtered.shape

(12354575, 4)

In [626]:
df_previous_payments = df_payments_filtered.merge(df_previous_filtered[['SK_ID_CURR','SK_ID_PREV','EXPECTED_EAR','AMT_CREDIT']], on='SK_ID_PREV', how='inner')

In [627]:
df_previous_payments.shape

(12354575, 7)

In [628]:
df_previous_payments['AMT_CREDIT'] = df_previous_payments['AMT_CREDIT'].fillna(0)
df_previous_payments['DAYS_INSTALMENT'] = df_previous_payments['DAYS_INSTALMENT'].fillna(0)
df_previous_payments['DAYS_ENTRY_PAYMENT'] = df_previous_payments['DAYS_ENTRY_PAYMENT'].fillna(0)
df_previous_payments['AMT_INSTALMENT'] = df_previous_payments['AMT_INSTALMENT'].fillna(0)

In [629]:
df_previous_payments = df_previous_payments\
.join((df_previous_payments.groupby('SK_ID_PREV')['DAYS_INSTALMENT']\
.agg(['min'])-30).rename(columns={'min':'START_DATE'}), on='SK_ID_PREV')

In [630]:
df_previous_payments = df_previous_payments\
.join(df_previous_payments.groupby('SK_ID_PREV')['DAYS_ENTRY_PAYMENT']\
.agg(['max']).rename(columns={'max':'END_DATE'}), on='SK_ID_PREV')

In [631]:
df_previous_payments['INDEX'] = df_previous_payments['DAYS_ENTRY_PAYMENT'] - df_previous_payments['START_DATE']

In [632]:
df_previous_payments[df_previous_payments['SK_ID_PREV'] == 2604725]

,SK_ID_PREV,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,SK_ID_CURR,EXPECTED_EAR,AMT_CREDIT,START_DATE,END_DATE,INDEX
1459036,2604725,-1094.0,-1126.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,118.0
1459037,2604725,-764.0,-763.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,481.0
1459038,2604725,-914.0,-945.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,299.0
1459039,2604725,-1064.0,-1064.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,180.0
1459040,2604725,-1184.0,-1185.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,59.0
1459041,2604725,-734.0,-740.0,642481.83,100042,0.281528,978408.0,-1244.0,-740.0,504.0
1459042,2604725,-1154.0,-1155.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,89.0
1459043,2604725,-824.0,-854.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,390.0
1459044,2604725,-794.0,-822.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,422.0
1459045,2604725,-1094.0,-1092.0,38934.18,100042,0.281528,978408.0,-1244.0,-740.0,152.0


In [633]:
def get_dict(df):
    keys = df['INDEX'].tolist()
    values = df['AMT_INSTALMENT'].tolist()
    dictionary = dict(zip(keys, values))
    
    return dictionary

In [634]:
df_previous_payments_t = df_previous_payments.groupby(['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE', 'AMT_CREDIT'])\
.progress_apply(lambda x: get_dict(x))

In [635]:
df_previous_payments_t = df_previous_payments_t.reset_index()

In [636]:
df_previous_payments_t.columns = ['SK_ID_CURR','SK_ID_PREV','START_DATE','END_DATE','AMT_CREDIT', 'TMP_EAR_DICT']

In [638]:
df_previous_payments_t.head(5)

,SK_ID_CURR,SK_ID_PREV,START_DATE,END_DATE,AMT_CREDIT,TMP_EAR_DICT
0,100001,1369693,-1739.0,-1628.0,23787.0,"{24.0: 3951.0, 111.0: 17397.9, 79.0: 3951.0}"
1,100002,1038818,-595.0,-49.0,179055.0,"{462.0: 9251.775, 220.0: 9251.775, 546.0: 5309..."
2,100003,1810518,-746.0,-544.0,1035882.0,"{56.0: 98356.995, 85.0: 98356.995, 202.0: 5608..."
3,100003,2396755,-2340.0,-1985.0,68053.5,"{114.0: 6737.31, 269.0: 6737.31, 141.0: 6737.3..."
4,100003,2636178,-827.0,-661.0,348637.5,"{137.0: 64567.665, 21.0: 64567.665, 108.0: 645..."


In [639]:
df_previous_payments_t.shape

(958905, 6)

In [647]:
df_previous_payments_t.head(5)

,SK_ID_CURR,SK_ID_PREV,START_DATE,END_DATE,AMT_CREDIT,TMP_EAR_DICT
0,100001,1369693,-1739.0,-1628.0,23787.0,"{24.0: 3951.0, 111.0: 17397.9, 79.0: 3951.0}"
1,100002,1038818,-595.0,-49.0,179055.0,"{462.0: 9251.775, 220.0: 9251.775, 546.0: 5309..."
2,100003,1810518,-746.0,-544.0,1035882.0,"{56.0: 98356.995, 85.0: 98356.995, 202.0: 5608..."
3,100003,2396755,-2340.0,-1985.0,68053.5,"{114.0: 6737.31, 269.0: 6737.31, 141.0: 6737.3..."
4,100003,2636178,-827.0,-661.0,348637.5,"{137.0: 64567.665, 21.0: 64567.665, 108.0: 645..."


In [651]:
# look at the exceptions
def get_new_dict(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT):
    credit = AMT_CREDIT
    period_length = int(END_DATE - START_DATE)
    d = TMP_EAR_DICT
    if period_length > 0:
        cf = [0] * (period_length + 1)
        cf[0] = -credit
        for index, amount in d.items():
            try:
                cf[int(index)] = amount
            except Exception as e:
                print(SK_ID_CURR, SK_ID_PREV, START_DATE, END_DATE, AMT_CREDIT, TMP_EAR_DICT)
    else:
        return [0]
    return cf

In [652]:
df_previous_payments_tt = df_previous_payments_t\
.progress_apply(lambda x: get_new_dict(x.SK_ID_CURR, x.SK_ID_PREV, x.START_DATE, x.END_DATE, x.AMT_CREDIT, x.TMP_EAR_DICT), axis=1)

108966 2805403 -170.0 -159.0 47290.5 {-20.0: 8744.94, 11.0: 33455.655, 0.0: 8744.94}
123458 2755582 -1838.0 -1787.0 30195.0 {-274.0: 5652.495, 0.0: 5652.495, 51.0: 16403.67, -239.0: 5652.495}
123458 2755582 -1838.0 -1787.0 30195.0 {-274.0: 5652.495, 0.0: 5652.495, 51.0: 16403.67, -239.0: 5652.495}
125014 1181029 -1177.0 -1163.0 32184.0 {0.0: 3858.84, -22.0: 3858.84, 14.0: 27725.355}
152554 1377212 -444.0 -443.0 17347.5 {-20.0: 3567.015, 1.0: 15827.58}
163044 2346107 -617.0 -572.0 148365.0 {45.0: 144025.785, -67.0: 10704.555}
171336 2049507 -938.0 -881.0 24682.5 {57.0: 4361.4, 27.0: 4361.4, -98.0: 4361.4}
174087 1214702 -1284.0 -1268.0 18351.0 {0.0: 3993.39, -9.0: 3993.39, -23.0: 3993.39, 16.0: 3990.105}
188263 2617776 -1220.0 -1209.0 26284.5 {11.0: 22766.985, -20.0: 3196.215, 0.0: 3196.215}
192645 1850683 -820.0 -798.0 15246.0 {22.0: 7864.515, -46.0: 3964.095, 0.0: 3964.095}
195834 2044431 -719.0 -634.0 67828.5 {-177.0: 12285.36, -173.0: 12285.36, 85.0: 47251.44}
195834 2044431 -719.0 

In [653]:
df_previous_payments_tt = df_previous_payments_tt.to_frame()

In [654]:
df_previous_payments_tt.columns = ['CF']

In [655]:
df_previous_payments_tt_j = df_previous_payments_t.join(df_previous_payments_tt, on=None, how='inner')

In [657]:
df_previous_payments_tt_j.shape

(958905, 7)

In [658]:
d = {288.0: 10469.34,
 78.0: 10602.45,
 43.0: 10602.45,
 126.0: 10602.45,
 229.0: 10602.45,
 195.0: 10602.45,
 167.0: 10602.45,
 13.0: 10602.45,
 251.0: 10602.45,
 92.0: 10602.45}

In [659]:
for key, value in d.items():
    print (int(key), value)

288 10469.34
78 10602.45
43 10602.45
126 10602.45
229 10602.45
195 10602.45
167 10602.45
13 10602.45
251 10602.45
92 10602.45


In [660]:
def get_actual_ear(cf):
    """ derives actual EAR based on all payments based on previous application
    """
    irr = np.irr(cf)
    ear = (1+irr)**360 - 1 # we take 1 year to be 360 days
    return ear

In [661]:
df_actual_ear = df_previous_payments_tt_j\
.progress_apply(lambda x: get_actual_ear(x.CF), axis=1)

KeyboardInterrupt: 

In [573]:
df_final = df_actual_ear.to_frame()

In [574]:
df_final.columns = ['ACTUAL_EAR']

In [575]:
df_final

,ACTUAL_EAR
0,0.293995
1,0.524947
2,0.802244
3,0.594947
4,1.360314
5,0.513768
6,0.287169


In [577]:
df_final_j = df_previous_payments_tt_j[['SK_ID_CURR','SK_ID_PREV']]\
.join(df_final, on=None, how='inner')

In [578]:
df_final_j

,SK_ID_CURR,SK_ID_PREV,ACTUAL_EAR
0,100042,1125423,0.293995
1,100042,1643959,0.524947
2,100042,1643964,0.802244
3,100042,1716930,0.594947
4,100042,2137382,1.360314
5,100042,2535181,0.513768
6,100042,2604725,0.287169


In [579]:
#df_actual_ear.columns = ['SK_ID_CURR', 'SK_ID_PREV', 'ACTUAL_EAR']

In [580]:
#df_actual_ear

In [581]:
df_final_j = df_final_j.groupby('SK_ID_CURR')['ACTUAL_EAR']\
.agg(['mean','median','std'])\
.rename(columns={'mean':'MEAN_ACTUAL_EAR','median':'MEDIAN_ACTUAL_EAR', 'std':'STD_ACTUAL_EAR'})

In [582]:
df_final_j

,MEAN_ACTUAL_EAR,MEDIAN_ACTUAL_EAR,STD_ACTUAL_EAR
SK_ID_CURR,,,
100042,0.625341,0.524947,0.369478


In [583]:
df_expected_actual = df_final_j.join(df_expected_ear, on='SK_ID_CURR', how='inner')

In [584]:
df_expected_actual

,MEAN_ACTUAL_EAR,MEDIAN_ACTUAL_EAR,STD_ACTUAL_EAR,MEAN_EXPECTED_EAR,MEDIAN_EXPECTED_EAR,STD_EXPECTED_EAR
SK_ID_CURR,,,,,,
100042,0.625341,0.524947,0.369478,0.404926,0.413872,0.222628


In [665]:
df_expected_ear.to_csv('/ws_nparigi/01_Work/04_mba/Year2/Block1/valuation/datascience-assignment/home-credit-default-risk/expected_ear.csv', index=True)